In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 100     # number of variables
num_ineq = 100    # number of constraints
num_data = 10000  # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [5]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msNonconvex
model = msNonconvex(num_var, num_ineq, timelimit=60)

In [7]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    try:
        xval, objval = model.solve("scip")
        # eval
        params.append(list(b))
        sols.append(list(list(xval.values())[0].values()))
        objvals.append(objval)
        conviols.append(sum(model.cal_violation()))
    except:
        params.append(list(b))
        sols.append(None)
        objvals.append(None)
        conviols.append(None)
    tock = time.time()
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
print("Number of None values: ", df["Sol"].isna().sum())
df.to_csv("result/nc_exact_100-100.csv")

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

ERROR: evaluating object as numeric value: x[0]
        (object: <class 'pyomo.core.base.var.VarData'>)
    No value for uninitialized NumericValue object x[0]
ERROR: evaluating object as numeric value: obj
        (object: <class 'pyomo.core.base.objective.ScalarObjective'>)
    No value for uninitialized NumericValue object x[0]


  1%|▊                                                                               | 1/100 [00:59<1:37:55, 59.35s/it]

ERROR: evaluating object as numeric value: x[0]
        (object: <class 'pyomo.core.base.var.VarData'>)
    No value for uninitialized NumericValue object x[0]
ERROR: evaluating object as numeric value: obj
        (object: <class 'pyomo.core.base.objective.ScalarObjective'>)
    No value for uninitialized NumericValue object x[0]


  2%|█▌                                                                              | 2/100 [01:59<1:37:31, 59.70s/it]

...

ERROR: Solver (scip) returned non-zero return code (4294967295)
ERROR: Solver log: WARNING: unknown parameter <quiet> SCIP version 8.1.0
[precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex
6.0.4] [GitHash: 6129793871] Copyright (c) 2002-2023 Zuse Institute Berlin
(ZIB)

    External libraries:
      Soplex 6.0.4         Linear Programming Solver developed at Zuse
      Institute Berlin (soplex.zib.de) [GitHash: 950b1658] CppAD 20180000.0
      Algorithmic Differentiation of C++ algorithms developed by B. Bell
      (github.com/coin-or/CppAD) ZLIB 1.2.13          General purpose
      compression library by J. Gailly and M. Adler (zlib.net) AMPL/MP
      4e2d45c4     AMPL .nl file reader library (github.com/ampl/mp) PaPILO
      2.1.4         parallel presolve for integer and linear optimization
      (github.com/scipopt/papilo) [GitHash: ee0677c4] bliss 0.77
      Computing Graph Automorphism Groups by T. Junttila and P. Kaski
      (https://users.aalto.fi/~tjuntti

 99%|██████████████████████████████████████████████████████████████████████████████▏| 99/100 [1:17:03<00:39, 39.11s/it]

ERROR: evaluating object as numeric value: x[0]
        (object: <class 'pyomo.core.base.var.VarData'>)
    No value for uninitialized NumericValue object x[0]
ERROR: evaluating object as numeric value: obj
        (object: <class 'pyomo.core.base.objective.ScalarObjective'>)
    No value for uninitialized NumericValue object x[0]


100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [1:18:02<00:00, 46.82s/it]


ValueError: All arrays must be of the same length

## Heuristic - Round

In [18]:
from src.heuristic import naive_round

In [20]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # relax
    model_rel = model.relax()
    # solve
    try:
        xval_rel, _ = model_rel.solve("scip")
        xval, objval = naive_round(xval_rel, model)
        # eval
        params.append(list(b))
        sols.append(list(list(xval.values())[0].values()))
        objvals.append(objval)
        conviols.append(sum(model_heur.cal_violation()))
    except:
        params.append(list(b))
        sols.append(None)
        objvals.append(None)
        conviols.append(None)
    tock = time.time()
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_heur_rnd_100-100.csv")

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

ERROR: evaluating object as numeric value: x[0]
        (object: <class 'pyomo.core.base.var.VarData'>)
    No value for uninitialized NumericValue object x[0]
ERROR: evaluating object as numeric value: obj
        (object: <class 'pyomo.core.base.objective.ScalarObjective'>)
    No value for uninitialized NumericValue object x[0]


100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [1:31:43<00:00, 55.03s/it]


ValueError: All arrays must be of the same length

## Learnable Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 256           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmNonconvex
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmNonconvex(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 3603.58
Epoch 1, Validation Loss: 7.49
Epoch 2, Validation Loss: 5.31
Epoch 3, Validation Loss: 4.32
Epoch 4, Validation Loss: 3.91
Epoch 5, Validation Loss: 3.57
Epoch 6, Validation Loss: 3.75
Epoch 7, Validation Loss: 3.59
Epoch 8, Validation Loss: 3.35
Epoch 9, Validation Loss: 3.27
Epoch 10, Validation Loss: 3.11
Epoch 11, Validation Loss: 3.08
Epoch 12, Validation Loss: 2.62
Epoch 13, Validation Loss: 2.65
Epoch 14, Validation Loss: 2.30
Epoch 15, Validation Loss: 2.30
Epoch 16, Validation Loss: 2.79
Epoch 17, Validation Loss: 2.60
Epoch 18, Validation Loss: 2.21
Epoch 19, Validation Loss: 2.43
Epoch 20, Validation Loss: 2.53
Epoch 21, Validation Loss: 2.68
Epoch 22, Validation Loss: 2.12
Epoch 23, Validation Loss: 2.36
Epoch 24, Validation Loss: 1.72
Epoch 25, Validation Loss: 1.81
Epoch 26, Validation Loss: 1.64
Epoch 27, Validation Loss: 1.78
Epoch 28, Validation Loss: 1.62
Epoch 29, Validation Loss: 1.87
Epoch 30, Validation Loss: 1.79
Epoch 31, Valid

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_lr_100-100.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:04<00:00, 23.11it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean     0.974443          0.000321      0.003584
std      0.090307          0.003211      0.001102
min      0.926433          0.000000      0.001999
25%      0.928231          0.000000      0.002998
50%      0.928231          0.000000      0.003416
75%      1.028227          0.000000      0.004227
max      1.438176          0.032107      0.007854
Number of infeasible solution: 1


## Learnable Threshold

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 256           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmNonconvex
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmNonconvex(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 2485.01
Epoch 1, Validation Loss: 3.66
Epoch 2, Validation Loss: 3.07
Epoch 3, Validation Loss: 3.04
Epoch 4, Validation Loss: 3.60
Epoch 5, Validation Loss: 1.93
Epoch 6, Validation Loss: 2.07
Epoch 7, Validation Loss: 2.43
Epoch 8, Validation Loss: 1.92
Epoch 9, Validation Loss: 2.12
Epoch 10, Validation Loss: 1.61
Epoch 11, Validation Loss: 2.26
Epoch 12, Validation Loss: 2.00
Epoch 13, Validation Loss: 1.95
Epoch 14, Validation Loss: 2.21
Epoch 15, Validation Loss: 2.14
Epoch 16, Validation Loss: 1.95
Epoch 17, Validation Loss: 2.22
Epoch 18, Validation Loss: 2.28
Epoch 19, Validation Loss: 1.58
Epoch 20, Validation Loss: 1.77
Epoch 21, Validation Loss: 1.41
Epoch 22, Validation Loss: 1.66
Epoch 23, Validation Loss: 2.14
Epoch 24, Validation Loss: 1.46
Epoch 25, Validation Loss: 1.87
Epoch 26, Validation Loss: 1.36
Epoch 27, Validation Loss: 1.56
Epoch 28, Validation Loss: 1.65
Epoch 29, Validation Loss: 1.83
Epoch 30, Validation Loss: 1.70
Epoch 31, Valid

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_lt_100-100.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 28.59it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean     1.037863          0.000355      0.003222
std      0.108651          0.003553      0.000864
min      0.929790          0.000000      0.001995
25%      0.980396          0.000000      0.002616
50%      1.014233          0.000000      0.003056
75%      1.073323          0.000000      0.003606
max      1.936775          0.035534      0.006984
Number of infeasible solution: 1


## Parametric Learning Then Rounding

In [17]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [18]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 256           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [19]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# build neuromancer problem for rounding
components = nn.ModuleList([smap]).to("cuda")
loss_fn = nmQuadratic(["b", "x"], num_var, num_ineq, penalty_weight)

In [20]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 2482.40
Epoch 1, Validation Loss: -5.52
Epoch 2, Validation Loss: -10.61
Epoch 3, Validation Loss: -11.78
Epoch 4, Validation Loss: -11.27
Epoch 5, Validation Loss: -12.69
Epoch 6, Validation Loss: -12.93
Epoch 7, Validation Loss: -12.88
Epoch 8, Validation Loss: -12.59
Epoch 9, Validation Loss: -13.56
Epoch 10, Validation Loss: -14.27
Epoch 11, Validation Loss: -13.90
Epoch 12, Validation Loss: -13.50
Epoch 13, Validation Loss: -14.48
Epoch 14, Validation Loss: -13.69
Epoch 15, Validation Loss: -13.29
Epoch 16, Validation Loss: -14.16
Epoch 17, Validation Loss: -13.76
Epoch 18, Validation Loss: -14.54
Epoch 19, Validation Loss: -15.16
Epoch 20, Validation Loss: -14.37
Epoch 21, Validation Loss: -14.57
Epoch 22, Validation Loss: -14.43
Epoch 23, Validation Loss: -14.97
Epoch 24, Validation Loss: -14.23
Epoch 25, Validation Loss: -14.97
Epoch 26, Validation Loss: -14.43
Epoch 27, Validation Loss: -15.15
Epoch 28, Validation Loss: -15.41
Epoch 29, Validation Los

In [21]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_pr_100-100.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:04<00:00, 22.40it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean    19.652664          0.201050      0.000987
std      0.588402          0.262282      0.000514
min     18.553989          0.000000      0.000000
25%     19.225016          0.006625      0.000999
50%     19.591134          0.098692      0.001001
75%     20.058785          0.287999      0.001082
max     21.196766          1.256986      0.002561
Number of infeasible solution: 78


### STE Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 256           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundSTEModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
rnd = roundSTEModel(param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], int_ind=model.int_ind, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 2485.01
Epoch 1, Validation Loss: -0.09
Epoch 2, Validation Loss: -4.60
Epoch 3, Validation Loss: -7.00
Epoch 4, Validation Loss: -6.73
Epoch 5, Validation Loss: -7.82
Epoch 6, Validation Loss: -6.93
Epoch 7, Validation Loss: -9.12
Epoch 8, Validation Loss: -7.76
Epoch 9, Validation Loss: -8.33
Epoch 10, Validation Loss: -7.82
Epoch 11, Validation Loss: -8.14
Epoch 12, Validation Loss: -8.18
Epoch 13, Validation Loss: -8.56
Epoch 14, Validation Loss: -8.84
Epoch 15, Validation Loss: -9.27
Epoch 16, Validation Loss: -9.11
Epoch 17, Validation Loss: -8.64
Epoch 18, Validation Loss: -9.19
Epoch 19, Validation Loss: -8.97
Epoch 20, Validation Loss: -9.41
Epoch 21, Validation Loss: -9.14
Epoch 22, Validation Loss: -9.53
Epoch 23, Validation Loss: -9.08
Epoch 24, Validation Loss: -9.05
Epoch 25, Validation Loss: -8.52
Epoch 26, Validation Loss: -8.82
Epoch 27, Validation Loss: -9.08
Epoch 28, Validation Loss: -9.39
Epoch 29, Validation Loss: -9.17
Epoch 30, Validati

In [11]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_st_100-100.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:04<00:00, 23.47it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean    21.124191               0.0      0.001992
std      0.388949               0.0      0.000854
min     20.633654               0.0      0.000994
25%     20.789707               0.0      0.001472
50%     20.873754               0.0      0.002001
75%     21.530539               0.0      0.002203
max     21.818590               0.0      0.006127
Number of infeasible solution: 0
